In [1]:
import cv2 as cv
from matplotlib.image import imread
import matplotlib.pyplot as plt

In [2]:
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from skimage.transform import resize
from skimage import img_as_ubyte

In [3]:
import numpy as np

In [4]:
from ipywidgets import Image
from ipycanvas import Canvas

In [73]:
from PIL import Image as PILImage

In [58]:
import colorsys, glob
from shutil import copyfile

In [7]:
def resize_dims(width, height, min_size=500):
    if width < height:
        return (min_size, int((height / width) * min_size))
    return (int((width / height) * min_size), min_size)

In [8]:
def dom(path, n_clusters, min_size=500):
    im = imread(path)
    oh, ow = len(im), len(im[0])
    
    f = 2 / 3
    
    x1 = int((ow / 2) * (1 - f))
    x2 = int((ow / 2) * (1 + f))
    y1 = int((oh / 2) * (1 - f))
    y2 = int((oh / 2) * (1 + f))
    
    imr = im[x1:x2, y1:y2]
    h, w = len(imr), len(imr[0])
    
    width, height = resize_dims(w, h, min_size=min_size)
    res_imr = img_as_ubyte(resize(imr, (height, width)))
    fimr = res_imr.reshape((width*height, 3))
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(fimr)
    clus = kmeans.cluster_centers_
    
    width, height = resize_dims(ow, oh, min_size=min_size)
    res_im = img_as_ubyte(resize(im, (height, width)))
    fim = res_im.reshape((width*height, 3))
    
    preds = kmeans.predict(fim)
    
    labels, counts = np.unique(preds, return_counts=True)
    
    clusters = []
    for i in range(len(labels)):
        label = labels[i]
        count = counts[i]
        color = tuple(map(int, clus[label]))
        clusters.append((color, count))

    return clusters, preds, width, height

In [9]:
def dom_show(path, n_clusters, min_size=500):
    clusters, preds, width, height = dom(path, n_clusters, min_size)
    
    img = Image.from_file(path)

    h = 500
    w = (width / height) * h

    canvas = Canvas(width=w*3, height=h)
    canvas.draw_image(img, 0, 0, w, h)

    y = 0
    for color, count in clusters:
        color = "#%02x%02x%02x" % color
        hh = (count / (width * height)) * h

        canvas.fill_style = color
        canvas.fill_rect(w, y, w, hh)

        y += hh
    
    preds_x = np.array(list(map(lambda x: np.array(clusters[x][0], dtype=np.uint8), preds)))
    pred_img = preds_x.reshape(height, width, 3)
    pred_img_file = PILImage.fromarray(pred_img, 'RGB')
    fname = path.split('/')[-1]
    pred_fname = f'img0/{fname}'
    pred_img_file.save(pred_fname)
    
    img0 = Image.from_file(pred_fname)
    canvas.draw_image(img0, 2 * w, 0, w, h)

    return canvas

In [10]:
def dom_show_by_value(path, n_clusters, min_size=500):
    clusters, preds, width, height = dom(path, n_clusters, min_size)
    
    img = Image.from_file(path)

    h = 500
    w = (width / height) * h

    canvas = Canvas(width=w*2, height=h)
    canvas.draw_image(img, 0, 0, w, h)
        
    modified_clusters = []
    max_value = 0
    max_value_color = [0, 0, 0]
    max_value_color_count = 0
    max_count = 0
    max_count_color = [0, 0, 0]
    for color, count in clusters:
        _1, _2, v = colorsys.rgb_to_hsv(*[x/255 for x in color])
        if v > 0.33:
            modified_clusters.append((color, count))
            if count > max_count:
                max_count = count
                max_count_color = color
        if v >= max_value:
            max_value = v
            max_value_color = color
            max_value_color_count = count
            
    if len(modified_clusters) == 0:
        modified_clusters = [(max_value_color, max_value_color_count)]
        max_count = max_value_color_count
        max_count_color = max_value_color
        
    total_count = sum([x[1] for x in modified_clusters])
        
    y = 0
    for color, count in modified_clusters:
        color = "#%02x%02x%02x" % color
        hh = (count / total_count) * h

        canvas.fill_style = color
        canvas.fill_rect(w, y, w, hh)

        y += hh
    
    return canvas

In [24]:
dom_show_by_value('img/2016-05-29 18.08.35 1260853863444241911_1514184470.jpg', 3)

Canvas(width=1000)

In [27]:
def dom_hue(path, n_clusters, min_size=500):
    clusters, preds, width, height = dom(path, n_clusters, min_size)
        
    modified_clusters = []
    max_value = 0
    max_value_color = [0, 0, 0]
    max_value_color_count = 0
    max_count = 0
    max_count_color = [0, 0, 0]
    for color, count in clusters:
        _1, _2, v = colorsys.rgb_to_hsv(*[x/255 for x in color])
        if v > 0.33:
            modified_clusters.append((color, count))
            if count > max_count:
                max_count = count
                max_count_color = color
        if v >= max_value:
            max_value = v
            max_value_color = color
            max_value_color_count = count
            
    if len(modified_clusters) == 0:
        max_count_color = max_value_color
    
    total_count = sum([x[1] for x in modified_clusters])
        
    h, s, v = colorsys.rgb_to_hsv(*[x/255 for x in max_count_color])
    return h, s, v

In [28]:
dom_hue('img\\2016-05-29 18.08.35 1260853863444241911_1514184470.jpg', 3)

(0.0, 0.0, 0.8941176470588236)

In [14]:
files = glob.glob("img/*.jpg")

In [53]:
file_colors = []
i = 0
for file in files[::-1]:
    h, s, v = dom_hue(file, 3)
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    file_colors.append((file, (r, g, b)))
    
#     img = imread(file)
    
#     f, axarr = plt.subplots(1, 2)
#     axarr[0].imshow(img)
#     axarr[0].axis('off')
#     axarr[1].imshow([[(r, g, b)]])
#     axarr[1].axis('off')
#     plt.show()
    
    i += 1
    print(i, end=', ')
    
#     if i > 20:
#         break
    

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222

In [63]:
import math
def step(r, g, b, repetitions=1):
    lum = math.sqrt(.241 * r + .691 * g + .068 * b)
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    
    h2 = int(h * repetitions)
    lum2 = int(lum * repetitions)
    v2 = int(v * repetitions)
    return (h2, lum, v2)

file_colors.sort(key=lambda x:step(x[1][0], x[1][1], x[1][2], 8))

In [ ]:
for file, color in file_colors:
    img = imread(file)
    r, g, b = color
    
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(img)
    axarr[0].axis('off')
    axarr[1].imshow([[(r, g, b)]])
    axarr[1].axis('off')
    plt.show()
    

In [65]:
i = 0
for file, color in file_colors:
    fname = file.split('\\')[-1]
    ofname = f'out/{i:03}.jpg'
    i += 1
    copyfile(file, ofname)

In [89]:
file_list = [file for file, color in file_colors]

def create_collage(width, height, listofimages):
    cols = 20
    rows = 25
    ts = 100
    size = ts, ts
    new_im = PILImage.new('RGB', (width, height))
    ims = []
    for p in listofimages:
        im = PILImage.open(p)
        w, h = im.size
        cw, ch = resize_dims(w, h, ts)
        im = im.resize((cw, ch), PILImage.ANTIALIAS)
        if w < h:
            l = 0
            r = ts
            t = ch / 2 - ts / 2
            b = ch / 2 + ts / 2
        else:
            t = 0
            b = ts
            l = cw / 2 - ts / 2
            r = cw / 2 + ts / 2
        im = im.crop((l, t, r, b))
        ims.append(im)
    i = 0
    x = 0
    y = 0
    for row in range(rows):
        for col in range(cols):
            new_im.paste(ims[i], (x, y))
            i += 1
            x += ts
        y += ts
        x = 0

    new_im.save('collage.jpg')

create_collage(2000, 2500, file_list)